Initialize workflow.

In [ ]:
import struct, socket
import shutil
import numpy as np
import pandas as pd
import linecache, bisect
import csv
import operator
import os
try:
    import ipywidgets as widgets # For jupyter/ipython >= 1.4
except ImportError:
    from IPython.html import widgets

from IPython.display import display, Javascript, clear_output

with open('/etc/duxbay.conf') as conf:
    for line in conf.readlines():
        if "LUSER" in line: LUSER = line.split("=")[1].strip('\n').replace("'","");  break

path = os.getcwd().split("/")
t_date = path[len(path)-1]  
spath = LUSER + '/ipython/user/' + t_date + '/'
sconnect = spath + 'flow_scores.csv'
sconnectbu = spath + 'flow_scores_bu.csv'
tmpconnect = sconnect +'.tmp'
stemp = sconnect + '.new'
coff = 150;
nwloc = contxtpath + 'networkcontext.txt' 
srcdict,srclist = {},[]
dstdict,dstlist = {},[]
sportdict,sportlist = {},[]
dportdict,dportlist = {},[]

In [ ]:
#load LDA scores#update lda doc with scores - either one edge,# or ALL that contain x IP and y port
def displaythis() :
    # build dict of IP addresses
    #sev,score, tstart,srcIP,dstIP,sport,dport,proto,ipkt,ibyt
    srcdict,srclist = {},[]
    dstdict,dstlist = {},[]
    sportdict,sportlist = {},[]
    dportdict,dportlist = {},[]
    srclist.append('- Select -')
    dstlist.append('- Select -')
    sportlist.append('- Select -')
    dportlist.append('- Select -')
    set_rules()
    with open(sconnect, 'r') as f:
        reader = csv.reader(f,delimiter=',') 
        reader.next();
        rowct = 1
        for row in reader:
            if row[2] not in srcdict and row[0] != '1':
                srclist.append(row[2])
                srcdict[row[2]] = struct.unpack("!L", socket.inet_aton(row[2]))[0]
            if row[3] not in dstdict and row[0] != '1':
                dstlist.append(row[3])
                dstdict[row[3]] = struct.unpack("!L", socket.inet_aton(row[3]))[0]
            if row[4] not in sportdict and row[0] != '1':
                sportlist.append(row[4])
                sportdict[row[4]] = row[4]
            if row[5] not in dportdict and row[0] != '1':
                dportlist.append(row[5])
                dportdict[row[5]] = row[5]
            if rowct == coff:
                break;
            rowct += 1    
    searches = widgets.Box(width=550,height=200)
    searchp = widgets.HBox(width=550,height=150)
    actions = widgets.Box(width=50,height=150)
    srcselect = widgets.Select(options=srclist,height=min(len(srclist)*12,150),width=130)
    dstselect = widgets.Select(options=dstlist,height=min(len(dstlist)*12,150),width=130)
    sportselect = widgets.Select(options=sportlist,height=min(len(sportlist)*12,150),width=90)
    dportselect = widgets.Select(options=dportlist,height=min(len(dportlist)*12,150),width=90)
    tableHTML = ("<table width='100%'><tr><td>" +
    "<td>Source IP:</td><td>Dest IP:</td><td>Src Port</td><td>Dst Port:</td></tr></table>")
    srctext = widgets.Text(value='',width=150, placeholder='Quick IP scoring')
    toplabel = widgets.HTML(value=tableHTML,width=550)
    ratingbut = widgets.RadioButtons(description='Rating:',options=['1', '2', '3'],width=50)
    assignbut = widgets.Button(description='Assign')
    updatebut = widgets.Button(description='Update')
    actions.children = [srctext,ratingbut,assignbut,updatebut]
    searchp.children = [srcselect,dstselect,sportselect,dportselect,actions]
    searches.children = [toplabel,searchp]
    display(searches)
    
    def update_sconnects(b):        
        clear_output()
        #define logic based on combo of input
        #Gets input values
        if srctext.value != '':
            svals = [srctext.value,dstselect.value,sportselect.value,dportselect.value]
        else:
            svals = [srcselect.value,dstselect.value,sportselect.value,dportselect.value]
        risk = ratingbut.value
        shash = 0
        fhash = [2,3,4,5]
        for k in xrange(len(svals)):
            if svals[k] == '- Select -': svals[k] = ''
            if svals[k] != '':
                shash += 2**k    
        with open(tmpconnect,'w') as g:
            riter = csv.writer(g,delimiter=',')
            riter.writerow(["sev"]+["tstart"]+["srcIP"]+["dstIP"]+["sport"]+["dport"]+["proto"]+["flag"]+["ipkt"]+["ibyt"]+
                           ["lda_score"]+["rank"]+["srcIpInternal"]+["destIpInternal"]+["srcGeo"]+["dstGeo"]+["srcDomain"]+
                           ["dstDomain"]+ ["gtiSrcRep"]+["gtiDstRep"]+["norseSrcRep"] +["norseDstRep"])
            with open(sconnect, 'r') as f:
                reader = csv.reader(f,delimiter=',')
                reader.next();
                rowct = 0
                #sev,tstart,srcIP,dstIP,sport,dport,proto,flag,ipkt,ibyt,lda_score,rank,srcIpInternal,destIpInternal,srcGeo,
                #dstGeo,srcDomain,dstDomain,gtiSrcRep,gtiDstRep,norseSrcRep,norseDstRep
                for row in reader: 
                    result = 0
                    for n in xrange(0,len(svals)):
                        if (2**n & shash) > 0:  
                            if row[fhash[n]] == svals[n]:
                                result += 2**n 
                    if result == shash:
                        row[0] = risk
                        rowct += 1
                    riter.writerow(row) 
        print "{0} matching connections scored".format(rowct)
        shutil.copyfile(tmpconnect,sconnect)
            
    def savesort(b):         
        clear_output()
        with open(stemp,'w') as g:
            riter = csv.writer(g,delimiter=',')
            riter.writerow(["sev"]+["tstart"]+["srcIP"]+["dstIP"]+["sport"]+["dport"]+["proto"]+["flag"]+["ipkt"]+["ibyt"]+
                           ["lda_score"]+["rank"]+["srcIpInternal"]+["destIpInternal"]+["srcGeo"]+["dstGeo"]+["srcDomain"]+
                           ["dstDomain"]+ ["gtiSrcRep"]+["gtiDstRep"]+["norseSrcRep"] +["norseDstRep"])
            reader = csv.reader(open(sconnect), delimiter=",")
            reader.next()
            srtlist = sorted(reader, key=lambda x: (int(x[0]), float(x[9])))
            riter.writerows(srtlist)
                
        shutil.copyfile(stemp,sconnect)
        print "Suspicious connects successfully updated"

    assignbut.on_click(update_sconnects)
    updatebut.on_click(savesort)

    
def set_rules():
    rops = ['leq','leq','leq','leq','leq','leq']
    rvals = ['','','',1024,'',54]
    risk = 2
    apply_rules(rops,rvals,risk)
    rops = ['leq','leq','leq','leq','eq','eq']
    rvals = ['','','',1024,3,152]
    risk = 2
    apply_rules(rops,rvals,risk)
    rops = ['leq','leq','leq','leq','eq','eq']
    rvals = ['','','',1024,2,104]
    risk = 2
    rops = ['leq','leq','eq','leq','leq','leq']
    rvals = ['','',0,1023,'','']
    risk = 2

    apply_rules(rops,rvals,risk)

    
def apply_rules(rops,rvals,risk):
    #define logic based on combo of input
    rhash = 0
    rfhash = [2,3,4,5,8,9]
    for k in xrange(len(rvals)):
        if rvals[k] != '':                
            rhash += 2**k
    with open(tmpconnect,'w') as g:
        riter = csv.writer(g,delimiter=',')
        riter.writerow(["sev"]+["tstart"]+["srcIP"]+["dstIP"]+["sport"]+["dport"]+["proto"]+["flag"]+["ipkt"]+["ibyt"]+
                       ["lda_score"]+["rank"]+["srcIpInternal"]+["destIpInternal"]+["srcGeo"]+["dstGeo"]+["srcDomain"]+
                       ["dstDomain"]+ ["gtiSrcRep"]+["gtiDstRep"]+["norseSrcRep"] +["norseDstRep"])
        
        with open(sconnect, 'r') as f:
            reader = csv.reader(f,delimiter=',')
            reader.next();
            rowct = 1
            for row in reader: 
                result = 0
                for n in xrange(0,6):
                    if (2**n & rhash) > 0:
                        if rops[n] == 'leq':
                            if int(row[rfhash[n]]) <= int(rvals[n]):
                                result += 2**n                           
                        if rops[n] == 'eq':
                            if int(row[rfhash[n]]) == int(rvals[n]):
                                result += 2**n                           
                if result == rhash:
                    row[0] = risk
                    rowct += 1
                riter.writerow(row) 
    shutil.copyfile(tmpconnect,sconnect)

    
def attack_heuristics():
    with open(sconnect, 'rb') as f:
        reader = csv.reader(f,delimiter=',') 
        reader.next();
        rowct = 1
        for row in reader:
            if row[2] not in srcdict:
                srcdict[row[2]] = row[2]
            if row[3] not in dstdict:
                 dstdict[row[3]] = row[3]
            if row[4] not in sportdict:
                sportdict[row[4]] = row[4]
            if row[5] not in dportdict:
                dportdict[row[5]] = row[5]

    df = pd.read_csv(sconnect)   
    gb = df.groupby([u'srcIP'])      
    
    for srcip in srcdict:
        try:
            if len(gb.get_group(srcip)) > 20:
                print srcip,'connects:',len(gb.get_group(srcip))
        except:
            print "Key Error for ip: " + srcip


Run attack heuristics.

In [ ]:
attack_heuristics() 

In [ ]:
set_rules()

In [ ]:
displaythis()

In [ ]:
# !cp $sconnectbu $sconnect